In [2]:
# to avoid errors on loading missingpy
import pandas as pd
import numpy as np
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
# With missforest we can use random forest to impute data. This is better than mean by far
from missingpy import MissForest
import lightgbm as lgb

# load stored data from EDA and cleaning Nan
%store -r data_frame_clean
train_df = data_frame_clean.copy()

## Label encoder to categorical columns

This is done so we can apply algorithms either to impute or to predict to our dataset

In [3]:
# taking a look to categorical variables
cat_vars = train_df.select_dtypes(exclude=[np.number])
cat_vars.describe()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,770,1379,1379,1379,1379,1460,1460,1460
unique,5,2,4,4,2,5,3,25,9,8,...,4,7,5,6,3,5,5,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal
freq,1151,1454,925,1311,1459,1052,1382,225,1260,1445,...,735,1360,380,870,605,1311,1326,1340,1267,1198


In [4]:
# process columns, apply LabelEncoder to categorical features
for c in cat_vars.columns:
    lbl = LabelEncoder() 
    lbl.fit(list(train_df[c].values)) 
    train_df[c] = lbl.transform(list(train_df[c].values))

## Impute categorical data with Random forest algorithm

In [5]:
# I need to get an example with real data.
# Make an instance and perform the imputation
imputer = MissForest(max_iter = 10, max_features=1.0, criterion='squared_error')
train_df_temp = train_df.drop('SalePrice', axis=1)
train_df_imputed = pd.DataFrame(imputer.fit_transform(train_df_temp))

# Renaming columns after imputation and add back the variable to predict
train_df_imputed = train_df_imputed.merge(train_df['SalePrice'], left_index = True, right_index= True)
train_df_imputed.columns = train_df.columns

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [6]:
%store train_df_imputed

Stored 'train_df_imputed' (DataFrame)
